#### Step 3: Model Fine-tuning

In [4]:
!pip install --upgrade sagemaker datasets

  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of pathos to determine which version is compatible with other requirements. This could take a while.
  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ----- ---------------------------------- 0.2/1.6 MB 6.3 MB/s eta 0:00:01
   --------------- ------------------------ 0.6/1.6 MB 7.7 MB/s eta 0:00:01
   ---------------------- ----------------- 0.9/1.6 MB 7.0 MB/s eta 0:00:01
   ------------------------------ --------- 1.2/1.6 MB 6.8 MB/s eta 0:00:

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'e:\\anaconda\\lib\\site-packages\\pyarrow\\_compute.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
model_id, model_version = "meta-textgeneration-llama-2-7b", "2.*"

In [ ]:
from sagemaker.jumpstart.estimator import JumpStartEstimator
import boto3

estimator = JumpStartEstimator(model_id=model_id,  environment={"accept_eula": "true"},instance_type = "ml.g5.2xlarge") 
estimator.set_hyperparameters(instruction_tuned="False", epoch="5")
estimator.fit({"training": f"s3://genaiwithawsproject2024/training-datasets/finance"})

#### Deploy the fine-tuned model


In [ ]:
finetuned_predictor = estimator.deploy()

#### Evaluate the pre-trained and fine-tuned model

In [4]:
def print_response(payload, response):
    print(payload["inputs"])
    print(f"> {response[0]['generation']}")
    print("\n==================================\n")

In [5]:
payload = {
    "inputs": "The investment tests performed indicate",
    "parameters": {
        "max_new_tokens": 64,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}
try:
    response = finetuned_predictor.predict(payload, custom_attributes="accept_eula=true")
    print_response(payload, response)
except Exception as e:
    print(e)

The investment tests performed indicate
'generation'


In [ ]:
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()